In [2]:
using Iterators
using DataStructures
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util")


4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util"                                                        

In [3]:
using Packing

In [4]:
function clean(ss)
    @pipe (ss 
    |> replace(_, r"\[.*?\] ?","")  #Remove Nonword sounds
    |> replace(_, r"\<.*?\> ","")  #Remove Verbal Deltions
    |> replace(_, r"\*(.*?)\*",s"\1") #Remove mispronounciation marks
    |> replace(_,r"\:|\-\s\.\s\-", "") #remove intraword pauses
    |> replace(_,r"\w+\- ","") #remove stuttered words
    |> replace(_, r"[!\.,\?]","")        #Remove punctation as it is not used traditionally (see sro spec)
    
    |> replace(_, r"\s+",' ') #Remove repeated spaces
    #|> replace(_, r"([A-Z])\s([A-Z])", s"\1\2") #Merge len(2) abbrev
    |> replace(_, r".*[~\(\)\-\<\>#'].*","")#Remove everything if anything unfixable found
    |> lowercase
    |> strip 
    )
end


clean (generic function with 1 method)

In [5]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

"**END**"

In [8]:
path="../../Resources/corpora/atis2_text/"
function valid(ss)
    typeof(ss) <: ASCIIString  && length(ss)>0
end
    
corpus = @pipe readdir(path)[1:4]  |> filter!(fn -> splitext(fn)[2]==".sro", _) |> map(_) do fn
    try open(readall, path*fn) end
    end |> filter!(valid,_) |> map(clean,_) |> filter!(valid,_) |> map(s->split(s),_);
corpus_vocab = @pipe corpus |> map(Set,_) |> reduce(union,_)
length(corpus_vocab)

14

In [9]:
using WordEmbeddings
LL, word_indexes, indexed_words = load_word2vec_embeddings("word_emb_data/GoogleNews-vectors-negative300.bin", length(corpus_vocab), corpus_vocab);

In [10]:
setdiff(corpus_vocab,indexed_words)


1-element Array{UTF8String,1}:
 "to"

In [11]:
#Kind of the opposite of a stop word. This word has little meaning (So zero value), but much structural importance
forcewords = ["and", "a", "of", "to"]
for word in forcewords
    @assert(!(word in indexed_words))
    push!(indexed_words, word)
    word_indexes[word] = length(indexed_words)
end
LL = [LL zeros(size(LL,1),length(forcewords))]

300x17 Array{Float64,2}:
  0.00374603   0.0746853    0.0348454    …  -0.0964167   0.0  0.0  0.0  0.0
 -0.0389198    0.0979106   -0.00010466       0.0181234   0.0  0.0  0.0  0.0
  0.0913317    0.0464506    0.0456797        0.0547328   0.0  0.0  0.0  0.0
  0.0120003    0.0498661    0.0319173        0.0739436   0.0  0.0  0.0  0.0
 -0.0705745   -0.062845     0.0150802       -0.026279    0.0  0.0  0.0  0.0
  0.105343    -0.112483     0.012518     …  -0.0349782   0.0  0.0  0.0  0.0
  0.0599364    0.0327887   -0.0415803       -0.104391    0.0  0.0  0.0  0.0
 -0.0573418   -0.110662    -0.0796467        0.0424088   0.0  0.0  0.0  0.0
  0.0381414    0.0409858   -0.0415803        0.0025033   0.0  0.0  0.0  0.0
  0.0110921    0.0281208    0.0983872        0.00602604  0.0  0.0  0.0  0.0
 -0.0653852   -0.0530539   -0.0629561    …  -0.0424088   0.0  0.0  0.0  0.0
 -0.0313953   -0.0710421   -0.0480223       -0.0366093   0.0  0.0  0.0  0.0
  0.0537093    0.0120111   -0.0325029       -0.131939    0.0  0

In [12]:
@assert(!(START_MARKER in indexed_words))
push!(indexed_words, START_MARKER)
word_indexes[START_MARKER] = length(indexed_words)

@assert(!(END_MARKER in indexed_words))
push!(indexed_words, END_MARKER)
word_indexes[END_MARKER] = length(indexed_words)
LL = [LL zeros(size(LL,1),2)]

300x19 Array{Float64,2}:
  0.00374603   0.0746853    0.0348454    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0389198    0.0979106   -0.00010466      0.0  0.0  0.0  0.0  0.0  0.0
  0.0913317    0.0464506    0.0456797       0.0  0.0  0.0  0.0  0.0  0.0
  0.0120003    0.0498661    0.0319173       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0705745   -0.062845     0.0150802       0.0  0.0  0.0  0.0  0.0  0.0
  0.105343    -0.112483     0.012518     …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0599364    0.0327887   -0.0415803       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0573418   -0.110662    -0.0796467       0.0  0.0  0.0  0.0  0.0  0.0
  0.0381414    0.0409858   -0.0415803       0.0  0.0  0.0  0.0  0.0  0.0
  0.0110921    0.0281208    0.0983872       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0653852   -0.0530539   -0.0629561    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0313953   -0.0710421   -0.0480223       0.0  0.0  0.0  0.0  0.0  0.0
  0.0537093    0.0120111   -0.0325029       0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                       

In [20]:
known_vocab = Set(indexed_words)
known_corpus = filter(corpus) do sent
    for word in sent
        if !( word in known_vocab)
            return false
        end
    end
    true
end;

In [ ]:
################LOADING Done, Now processing

In [ ]:
using DataStructures

function Base.sum(acc::Accumulator)
    sum(values(acc.map))
end

function Base.sum(acc::Dict)
    sum(values(acc))
end

function freq2prob{T,V<:Number}(acc::Union{Accumulator{T,V},Dict{T,V}})
    
    ret=Dict{T,Float64}()
    total = sum(acc)
    for (k,v) in acc
        ret[k]=v/total
    end
    ret
end

In [ ]:


function collect_grams_stats(sentences)
    unigrams = counter(AbstractString)
    bigrams = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        push!(bigrams[START_MARKER], sent[1])
        for ii in 1:length(sent)-1
            push!(unigrams,sent[ii])
            push!(bigrams[sent[ii]], sent[ii+1])    
        end
        push!(unigrams,sent[end])
        push!(bigrams[sent[end]], END_MARKER)
    end
    
    
    [k=>v.map for (k,v) in bigrams], unigrams.map
end




In [ ]:
using StatsBase
# modified from https://github.com/JoFrhwld/GoodTuring.jl/blob/master/GoodTuring.jl
function simpleGoodTuring(speciesCountDict::Dict)
    speciesCountVec = collect(values(speciesCountDict))
        
    totalCounts = sum(speciesCountVec)
    cofcDict = countmap(speciesCountVec)
    r = sort(collect(keys(cofcDict)))

    N = size(r,1)
    Nr = [cofcDict[r[i]] for i in 1:N]

    p0 = haskey(cofcDict, 1.0) ? cofcDict[1.0] / totalCounts : 0.0
    
    Z = sgtZ(r,Nr)
    logr = map(log,r)
    logZ = map(log,Z)

    X = hcat(ones(N), logr)
    Y = copy(logZ )
    coefs = X\Y
    intercept = coefs[1]
    slope = coefs[2]

    useY = false
    rSmooth = Array{Float64}(N)
    for i in 1:N
        @inbounds thisr = r[i]
        
        #y = ((thisr+1.0)^(slope+1.0))/(thisr^slope)
        #The above is the much simplified form of the below (Performance identical output differs by 10^-16)
        y = (thisr+1.0) * exp(slope * log(thisr+1.0) + intercept) / exp(slope * log(thisr) + intercept)

        if !in(thisr+1, r)
            useY = true
        end

        if useY
            rSmooth[i] = y
        else
            x = (thisr+1) * cofcDict[thisr + 1]/cofcDict[thisr]
            thisNr = cofcDict[thisr]
            thisNr1 = cofcDict[thisr+1]

            t = 1.96 * ((thisr+1)^2) * (thisNr1 / thisNr^2) * (1 + (thisNr1 / thisNr))

            if abs(x-y) > t
                @inbounds rSmooth[i] = x
            else
                useY = true
                @inbounds rSmooth[i] = y
            end
        end
    end

    smoothTot = sum(Nr.*rSmooth)
    sgtProb  = (1.0 - p0) .* (rSmooth/smoothTot)
    sgtProbDict = Dict([r[i] => sgtProb[i] for i in 1:N])
    sgtDict = Dict([sp=>sgtProbDict[speciesCountDict[sp]] for sp in keys(speciesCountDict)])

    sgtDict, sgtProbDict, p0
end


function sgtZ(r::Array, Nr::Array)
    j = r
    i = [0; j[1:end-1]]
    lastK = 2*j[end] - i[end]
    k = [j[2:end]; lastK]
    Float64[(2*Nr[iter])/(k[iter]-i[iter]) for iter = 1:length(j)]
end

function simpleGoodTuring(speciesCountVec::Vector)
    sgtD = simpleGoodTuring(Dict([ii=>v for (ii,vv) in enumerate(speciesCountVec)]))
    [sgtD[ii] for ii in 1:length(speciesCountVec)]
end

In [ ]:
function katz_bigrams(bigram_freq::Dict, unigrams_freq::Dict)
    k_bigrams = Dict()
    
    for first in keys(bigram_freq)
        smoothed,_,p0 = simpleGoodTuring(bigram_freq[first])
        k_bigrams[first] = smoothed
        
        backoff_keys = setdiff(keys(unigram_freq),keys(smoothed))
        #share the p0 proability mass between them
        total = sum([unigrams_freq[key] for key in backoff_keys])
        for second in backoff_keys
            k_bigrams[first][second]=p0.*unigrams_freq[second]./total
        end
    end
    k_bigrams
end

In [ ]:
function dict2mat(bigrams::Dict, word_indexes::Dict{AbstractString,Int64}, dense=False)
    mat  = (dense ? zeros: spzeros)(length(word_indexes),length(word_indexes))
    for first in keys(bigrams)
        for second in keys(bigrams[first])
            mat[word_indexes[second], word_indexes[first]] = bigrams[first][second]
        end
    end
    mat
end

In [ ]:
bigram_freq, unigram_freq = collect_grams_stats(known_corpus);
kbigrams=katz_bigrams(bigram_freq, unigram_freq)
kbigrams_mat = dict2mat(kbigrams,word_indexes,true)


In [ ]:
using Gadfly
using Distributions

In [ ]:
sent_lengths = map(length, known_corpus)
plot(x=sent_lengths, Geom.histogram)

In [ ]:
sent_length_dist = fit_mle(Gamma, sent_lengths)
plot(x=[round(rand(sent_length_dist)) for _ in 1:length(sent_lengths)], Geom.histogram)

In [ ]:
length_prob=cdf(sent_length_dist,[1.5:1.0:50.5])-cdf(sent_length_dist,[0.5:1.0:49.5])


In [ ]:
function collect_cooccur_stats(sentences)
    unioccur = counter(AbstractString)
    bioccur = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        for ii in 1:length(sent)
            push!(unioccur, sent[ii])
            for jj in 1:length(sent)
                if ii==jj
                    continue
                end
                push!(bioccur[sent[ii]], sent[jj])    
            end
        end       
    end
    
    [k=>v.map for (k,v) in bioccur], unioccur.map
    
end

In [ ]:
bioccur_freq, unioccur_freq = collect_cooccur_stats(known_corpus)

bioccur_mat = dict2mat(bioccur_freq,word_indexes,true)
bioccur_mat.+=1.0 # Add one smoothing
bioccur_mat./=sum(bioccur_mat)

unioccur = freq2prob(unioccur_freq)
unioccur_vec = Float64[word in keys(unioccurs) ? unioccurs[word] : 0.0 for word in indexed_words]

In [ ]:
unioccur_vec_smoothed = Float64[word in keys(unioccur_freq) ? unioccur_freq[word] : 0.0 for word in indexed_words]
unioccur_vec_smoothed.+=1.0 # Add one smoothing
unioccur_vec_smoothed./=sum(unioccur_vec_smoothed)

In [ ]:
sum(bioccur_mat,2)

In [ ]:
open("atis_data.jsz","w") do fh
    data = Dict([
        ("bigrams", kbigrams_mat),
        ("bioccur", bioccur_mat),
        ("unioccur", unioccur_vec),
        
        ("length_prob", length_prob),
        ("LL",LL),
        ("word_indexes", word_indexes),
        ("indexed_words", indexed_words),
        ])
    serialize(fh, data)    
end

In [ ]:
function likelyhood(sent, bigrams)
    words = split(sent)
    words = [START_MARKER; words; END_MARKER]
    
    p=1.0
    for ii in 1:length(words)-1
        p*=bigrams[words[ii]][words[ii+1]]
    end
    p
end


function select_word{S<:AbstractString,V}(unigrams::Dict{S,V})
    cutoff = rand()
    total = 0.0
    for next_word in keys(unigrams)
        total+=unigrams[next_word]
        if total>=cutoff
            return next_word
        end
    end
    assert(False, "Should never reach here") 
end

function random_walk(bigrams)
    words=[]
    cur = START_MARKER
    while(cur!=END_MARKER)
        cur = select_word(bigrams[cur])
        push!(words,cur)
    end
    words = words[1:end-1]
    join(words, " ")
end

walk =random_walk(kbigrams) 
print(walk*"\t")
print(likelyhood(walk,kbigrams))

In [ ]:
LL

Frame's Magic Mass-Sharing Co-occurance PMF, Inspired by Bengio 2003
---

In [ ]:
addprocs(11)

In [17]:

immutable LU_NN{N<:Number}
    C::Matrix{N} #lookup matrix
    H::Matrix{N}
    d::Vector{N}
    U::Matrix{N}
    b::Vector{N}
end


In [21]:
function prepare_cases(sentence)
    Task() do
        sentence_iis = Int[word_indexes[word] for word in sentence]
        for n_givens in 1:length(sentence_iis)
            given_prob = 2*1.0/(n_givens)
            given_keep = rand(length(sentence_iis)).<given_prob
            given_iis::Vector{Int} =  sentence_iis[given_keep]
            cooccur_iis::Vector{Int} =  sentence_iis[~given_keep]
            produce(given_iis, cooccur_iis)
        end   
    end
end

prepare_cases (generic function with 1 method)

In [22]:
training_case_type = Tuple{Vector{Int64},Vector{Int64}}
training_cases = @pipe (known_corpus
                            |> map(prepare_cases, _) |> chain(_...)
                            |> repeated(_, 5) |> chain(_...) 
                            |> collect(training_case_type,_))


28-element Array{Tuple{Array{Int64,1},Array{Int64,1}},1}:
 ([4,1,2,11,5,17,9,3,12,17,13],Int64[])
 ([4,1,2,11,5,17,9,3,12,17,13],Int64[])
 ([4,1,2,5,17,9,3,12,17],[11,13])      
 ([1,11,17,12],[4,2,5,9,3,17,13])      
 ([4,1,3],[2,11,5,17,9,12,17,13])      
 ([4,17,3],[1,2,11,5,9,12,17,13])      
 ([2],[4,1,11,5,17,9,3,12,17,13])      
 ([4,9,12],[1,2,11,5,17,3,17,13])      
 ([1],[4,2,11,5,17,9,3,12,17,13])      
 (Int64[],[4,1,2,11,5,17,9,3,12,17,13])
 ([4,11,17,12],[1,2,5,9,3,17,13])      
 ([10,7,17,6,3,12,17,13],Int64[])      
 ([10,7,17,6,3,12,17,13],Int64[])      
 ⋮                                     
 ([12],[10,7,17,6,3,17,13])            
 ([10,3,17],[7,17,6,12,13])            
 (Int64[],[10,7,17,6,3,12,17,13])      
 ([4,1,2,11,8,3,12,17,13],Int64[])     
 ([4,1,2,11,8,3,12,17,13],Int64[])     
 ([4,1,2,8,12],[11,3,17,13])           
 ([4,1,11,3,12,17],[2,8,13])           
 ([1,12],[4,2,11,8,3,17,13])           
 ([4,1,8,3,12,13],[2,11,17])           
 ([8,3,12],[4,1,2,11,1

In [ ]:
fi

In [62]:
function LU_NN(dEmb, dHidden, dOut)
    LU_NN(
        0.01*(randn((dEmb,dOut))), #C
        0.01*(randn((dHidden,dEmb))), #H
        0.01*(randn(dHidden)), #d
        0.01*(randn((dOut,dHidden))), #U
        0.01*(randn(dOut))#b
    )
end

LU_NN{N<:Number}

In [187]:
@everywhere function softmax(xs)
    numer = exp(xs)
    numer./sum(numer)
end


@everywhere function σ(xs)
    1.0./(1+exp(-xs))
end

#########################################



@everywhere function δ(δ_above, W)
    (W'*δ_above)
end

@everywhere function δ_tanh(δ_above, a)
    const dz = 1-a.^2
    δ_above.*dz
end

@everywhere function δ_softmax_ce(actual, expected)
    actual-expected
end

@everywhere function δ_σ(δ_above, a)
    const dz = a .- a.^2
    δ_above.*dz
end

@everywhere function δ_output_sq_loss(actual, expected) 
    -(expected-actual)
end

#############################################

@everywhere function sq_loss(actual, expected)
    0.5*sum((expected-actual).^2)
end

@everywhere function ce_loss(actual, expected)
    -sum(expected.*log(actual))
end

@everywhere function forward(x,nn::LU_NN)
    aa = tanh(nn.H*x+nn.d)
    out = softmax(nn.U*aa + nn.b)
    out, aa
end

@everywhere function feedforward_backprop(xx,nn::LU_NN, expected_output)
    actual_output, aa = forward(xx,nn)
    err = ce_loss(actual_output, expected_output)
    δ_top = δ_softmax_ce(actual_output, expected_output)
    ΔU  = δ_top*aa'
    Δb  = δ_top
    δ_hidden = δ_tanh(δ(δ_top, nn.U),aa)
    ΔH  = δ_hidden*xx'
    Δd  = δ_hidden
    δ_bottom = δ(δ_hidden, nn.H)
    Δx  = δ_bottom
    Δx,ΔH,Δd,ΔU,Δb,err

end

In [188]:
nn_outer = LU_NN(8, 16, length(indexed_words));

xx=sum([nn_outer.C[:,g_ii] for g_ii in [1,2,3]])
target = zeros(nn_outer.b)
target[[10,11,12]]=1.0/3


using ForwardDiff

function f(θ)
    x,H,d,U,b = unpack(θ,size(xx),(16,8),16,(19,16),19)
    nn =  LU_NN(0.0*H, H,d,U,b)#using 0.0*H as a dummy value for C
    
    actual, _=forward(x,nn)
    ce_loss(actual, target)
end

function calc_ag(θ)
    x,H,d,U,b = unpack(θ,size(xx),(16,8),16,(19,16),19)
    nn =  LU_NN(0.0*H, H,d,U,b) #using 0.0*H as a dummy value for C
    Δx,ΔH,Δd,ΔU,Δb,err = feedforward_backprop(x,nn, target)
end

g = ForwardDiff.gradient(f)

t=pack(xx,nn_outer.H,nn_outer.d,nn_outer.U,nn_outer.b)
a_x, a_H, a_d, a_U, a_b, a_err = calc_ag(t)
dg = g(t)  #Commented out so can't be run  
d_x, d_H, d_d, d_U, d_b = unpack(dg,size(xx),(16,8),16,(19,16),19)


@printval f(t) == a_err
@printval findmax(abs(d_x .- a_x))
@printval findmax(abs(d_H .- a_H))
@printval findmax(abs(d_d .- a_d))
@printval findmax(abs(d_U .- a_U))
@printval findmax(abs(d_b .- a_b))

f(t) == a_err = true
findmax(abs(d_x .- a_x)) = (1.0842021724855044e-19,3)
findmax(abs(d_H .- a_H)) = (1.0842021724855044e-19,12)
findmax(abs(d_d .- a_d)) = (3.469446951953614e-18,12)
findmax(abs(d_U .- a_U)) = (1.734723475976807e-18,69)
findmax(abs(d_b .- a_b)) = (5.551115123125783e-17,10)


In [169]:
@pz d_U
@pz nn_outer.U

d_U		Array{Float64,2}	(19,16)
nn_outer.U		Array{Float64,2}	(19,16)


In [163]:
@printval findmax(abs(nn_outer.H .- d_H))
@printval findmax(abs(nn_outer.d .- d_d))
@printval findmax(abs(nn_outer.U .- d_U))
@printval findmax(abs(nn_outer.b .- d_b))

findmax(abs(nn_outer.H .- d_H)) = (0.023995581845296828,72)
findmax(abs(nn_outer.d .- d_d)) = (0.02595172940401666,5)
findmax(abs(nn_outer.U .- d_U)) = (0.029180588881421714,61)
findmax(abs(nn_outer.b .- d_b)) = (0.293020737520177,10)


In [164]:
@everywhere function mysubarray(xs, id=myid(), nchunks=nworkers())
    len = length(xs)
    chunk_size = div(len, nchunks+1)
    start_index = (id-2)*chunk_size + 1
    end_index = start_index+chunk_size-1
    print(start_index : end_index)
    sub(xs, start_index : end_index)
end

@everywhere function train_one(given_iis, target_iis, C, W, b)
    given_sowe = length(given_iis)>0 ? sum([C[:,g_ii] for g_ii in given_iis]) : zeros(C[:,1])
    target = zeros(b) #just while we are testing use a one hot set rep
    for t_ii in target_iis
        target+=1.0/length(target_iis)
    end
        
    Δx,ΔW,Δb, err = feedforward_backprop(given_sowe,W,b, target)
    
    ΔC = zeros(C)
    for g_ii in given_iis
        @inbounds ΔC[:, g_ii]+=Δx
    end
    ΔC,ΔW,Δb, err
end


function train_all(training_cases,C, W, b)
    
    function accumulate_training_over(cases,fun)
        total_ΔC=zeros(C)
        total_ΔW=zeros(W)
        total_Δb=zeros(b)
        total_err = 0.0
        for case in cases
            ΔC, ΔW, Δb,err = fun(case)
            @inbounds total_ΔC+=ΔC
            @inbounds total_ΔW+=ΔW
            @inbounds total_Δb+=Δb
            total_err+=err
        end
        total_ΔC, total_ΔW, total_Δb, total_err
    end
    
    function train_remote()
        accumulate_training_over(mysubarray(training_cases),
                                 gt_iis -> train_one(gt_iis[1], gt_iis[2], C, W, b) )
    end
    
    r_updates = [@spawnat(id, train_remote())  for id in workers()]
        
    totals = accumulate_training_over(r_updates, fetch)
    ([tot./length(training_cases) for tot in totals]...)
end
    

train_all (generic function with 1 method)

In [ ]:
function uncached_loss_and_loss_grad!(θ::Vector, grad::Vector)    
    C, W, b = unpack!(θ, CC,WW,bb)
    ΔC, ΔW, Δb, err = train_all(training_cases, C, W, b )
    pack!(grad, ΔC, ΔW, Δb)
    err
end

_loss_and_loss_grad=Dict{Vector{Float64},Tuple{Float64, Vector{Float64}}}()
function loss_and_loss_grad!(θ::Vector, grad::Vector)    
    if haskey(_loss_and_loss_grad,θ)
        err, grad[:]= _loss_and_loss_grad[θ]
    else
        err = uncached_loss_and_loss_grad!(θ, grad)
        _loss_and_loss_grad[θ] = (err, copy(grad))
    end
    err
end

function loss!(θ::Vector)  
    dummy_grad = similar(θ) 
    loss_and_loss_grad!(θ, dummy_grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    loss_and_loss_grad!(θ, grad)
end


In [ ]:
using Optim
push!(LOAD_PATH, "../Optimisation")
using AdaDelta

In [ ]:
opt_func = DifferentiableFunction(loss!,loss_grad!,loss_and_loss_grad!)
θ=pack(CC,WW,bb)
#@time res = optimize(opt_func, θ, method=:l_bfgs, show_trace = true, store_trace = true, iterations = 10);
@time res = adadelta(opt_func, θ, show_trace = true, iterations = 500);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged                       
@printval res.f_converged 


In [ ]:
sowe = CC[:,word_indexes["ground"]]
ns = forward(sowe,WW,bb)
indexed_words[findmax(ns)[2]]

In [ ]:
@printval res.gr_converged
@printval res.x_converged                       
@printval res.f_converged 


In [ ]:
var(CC)

In [ ]:
var(bb)

In [ ]:
var(WW)